# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [38]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from cassandra.query import BatchStatement

#### Creating list of filepaths to process original event csv data files

In [39]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [40]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("Total number of rows:",len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# Counter for counting the number of empty rows skipped while inserting into 'event_datafile_new.csv'
skipped_rows = 0
file_name = 'event_datafile_new.csv'

with open(file_name, 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            skipped_rows += 1
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

print("Skipped rows:", skipped_rows)

Total number of rows: 8056
Skipped rows: 1236


In [41]:
# check the number of rows in your csv file
with open(file_name, 'r', encoding = 'utf8') as f:
    print("Total number of rows:",sum(1 for line in f))
    

Total number of rows: 6821


In [42]:
# Load csv to Pandas Dataframe
df = pd.read_csv(file_name) 
# Preview the first 5 lines of the loaded data 
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [43]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [44]:
# Documentation:
# https://docs.datastax.com/en/developer/python-driver/3.20/getting_started/
# https://docs.datastax.com/en/developer/python-driver/3.20/api/cassandra/query/

# Create a Keyspace 
KEYSPACE = "sparkify"

session.execute("""
                CREATE KEYSPACE IF NOT EXISTS %s
                WITH replication = { 'class': 'SimpleStrategy', 'replication_factor': '1' }
                """ % (KEYSPACE))

#### Set Keyspace

In [45]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace(KEYSPACE)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [61]:
## Insert function to replace loading of csv file everytime
def insert_into_table(insert_statement, df):
    # For each row in Dataframe, insert
    for index, row in df.iterrows():
        session.execute(insert_statement, tuple(row))

In [53]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
create_table1 = """
                CREATE TABLE IF NOT EXISTS session_items (
                    sessionId int,
                    itemInSession int,
                    artist text,
                    song_title text,
                    song_length float,
                    PRIMARY KEY (sessionId, itemInSession)
                )
                """
session.execute(create_table1)

In [58]:
# Insert statement for the table data
query1 = "INSERT INTO session_items (sessionId, itemInSession, artist, song_title, song_length)"
query1 = query1 + " values (%s, %s, %s, %s, %s)"
# Subset of the Dataframe corresponding to the insert statement
df1 = df[['sessionId','itemInSession','artist','song','length']]
# Insert data
insert_into_table(query1, df1)

In [24]:
# Replaced with insert_into_table function

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
#file = 'event_datafile_new.csv'
#
#with open(file, encoding = 'utf8') as f:
#    csvreader = csv.reader(f)
#    next(csvreader) # skip header
#    for line in csvreader:
#        ## Assign the INSERT statements into the `query` variable
#        query = "INSERT INTO session_items (sessionId, itemInSession, artist, song_title, song_length)"
#        query = query + " values (%s, %s, %s, %s, %s)"
#        
#        ## Assign which column element should be assigned for each column in the INSERT statement.
#        # Note:
#        # [bash]# head -n 2 event_datafile_new.csv 
#        # "artist","firstName","gender","itemInSession","lastName","length","level","location","sessionId","song","userId" <- headers
#        # "Stephen Lynch","Jayden","M","0","Bell","182.85669","free","Dallas-Fort Worth-Arlington, TX","829","Jim Henson's Dead","91" <- first row
#        artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId = line
#        
#        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
#        session.execute(query, (int(sessionId), int(itemInSession), artist, song, float(length)))

#### Do a SELECT to verify that the data have been inserted into each table

In [59]:
## Add in the SELECT statement to verify the data was entered into the table
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
select_table1 = """
                SELECT artist, song_title, song_length FROM session_items WHERE sessionId = 338 AND itemInSession = 4
                """
rows = session.execute(select_table1)

for row in rows:
    print(row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [62]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
create_table2 = """
                CREATE TABLE IF NOT EXISTS user_session_items (
                    userId int,
                    sessionId int,
                    itemInSession int,
                    artist text,
                    song_title text,
                    firstName text,
                    lastName text,
                    PRIMARY KEY (userId, sessionId, itemInSession)                  
                ) WITH CLUSTERING ORDER BY (sessionId ASC, itemInSession ASC)
                """
session.execute(create_table2)


# Insert statement for the table data
query2 = "INSERT INTO user_session_items (userId, sessionId, itemInSession, artist, song_title, firstName, lastName)"
query2 = query2 + " values (%s, %s, %s, %s, %s, %s, %s)"
# Subset of the Dataframe corresponding to the insert statement
df2 = df[['userId','sessionId','itemInSession','artist','song','firstName','lastName']]
# Insert data
insert_into_table(query2, df2)


## Add in the SELECT statement to verify the data was entered into the table
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
select_table2 = """
                SELECT artist, song_title, firstName, lastName  FROM user_session_items WHERE userid = 10 AND sessionid = 182
                """
rows = session.execute(select_table2)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [63]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
create_table3 = """
                CREATE TABLE IF NOT EXISTS song_users (
                    song_title text,
                    firstName text,
                    lastName text,
                    PRIMARY KEY (song_title)                  
                )
                """
session.execute(create_table3)


# Insert statement for the table data
query3 = "INSERT INTO song_users (song_title, firstName, lastName)"
query3 = query3 + " values (%s, %s, %s)"
# Subset of the Dataframe corresponding to the insert statement
df3 = df[['song','firstName','lastName']]
# Insert data
insert_into_table(query3, df3)


## Add in the SELECT statement to verify the data was entered into the table
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
select_table2 = """
                SELECT firstName, lastName FROM song_users WHERE song_title = 'All Hands Against His Own'
                """
rows = session.execute(select_table2)

for row in rows:
    print(row)

Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [4]:
## Drop the table before closing out the sessions

In [64]:
session.execute("DROP TABLE IF EXISTS session_items;")

session.execute("DROP TABLE IF EXISTS user_session_items;")

session.execute("DROP TABLE IF EXISTS song_users;")

### Close the session and cluster connection¶

In [65]:
session.shutdown()
cluster.shutdown()